In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prave\\Desktop\\Projects\\text_summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\prave\\Desktop\\Projects\\text_summarizer'

## WorkFlow for the given project
1. Update config.yaml
2. Update params.yaml
3. update entity
4. Update the configuration manager in src config
5. Update the componenets
6. Update the pipeline
7. Update the main.py
8. Update the app.py

In [5]:
# Updating Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
#Updation configManager inside src.config

from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


    

In [8]:
# Updating Componenets
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size
import requests

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    """
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")

    """
    def download_file(self):
        fname = "data.zip"
        if not os.path.exists(self.config.local_data_file):
            res = requests.get(self.config.source_URL)

            if res.status_code == 200:
                
                content = res.content

                with open (fname, "wb") as file:
                    file.write(content)
            logger.info(f"{fname} downloaded successfully...!")
        
        else:
            logger.info(f"{fname} already exists of size : {get_size(Path(self.config.local_data_file))}")


    def extract_zip(self):
        unzipped_path = self.config.unzip_dir
        os.makedirs(unzipped_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzipped_path)

In [17]:
# Creating Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2023-07-18 18:22:06,318: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-18 18:22:06,319: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-18 18:22:06,320: INFO: common: created directory at: artifacts]
[2023-07-18 18:22:06,322: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-18 18:22:06,322: INFO: 2652045047: data.zip already exists of size : ~7718 KB]
